#Text Summarization

The creation of a model that accurately summarizes a long piece of text.

# Import Necessary Modules

In [1]:
# Check runtime gpu
!nvidia-smi

Wed Aug 23 16:09:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install transformers
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 93.1 MB/s eta 0:00:

In [3]:
# More dependancies for distributed training
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.8 MB/s eta 0:00:00
Found existing installation: transformers 4.32.0
Uninstalling transformers-4.32.0:
  Successfully uninstalled transformers-4.32.0
Found existing installation: accelerate 0.22.0
Uninstalling accelerate-0.22.0:
  Successfully uninstalled accelerate-0.22.0
  Using cached transformers-4.32.0-py3-none-any.whl (7.5 MB)
  Using cached accelerate-0.22.0-py3-none-any.whl (251 kB)


In [4]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.0 MB/s eta 0:00:00


In [5]:
# Import the necessary modules
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
from tqdm import tqdm
import py7zr
import nltk
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# Set device to cuda
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
device

'cuda'

# Load Data, Tokenizer and Model

In [8]:
# Load data
dataset = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [9]:
# Load model and tokenizer

## Model
model_name = "google/pegasus-cnn_dailymail"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Prepare Data for Modelling

In [10]:
# Get Split lengths
split_lengths = list(dataset.num_rows.values())

In [11]:
# Create features from dataset
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

  target_encodings = tokenizer(text_target=example_batch['summary'], max_length = 128, truncation = True )

  return {
      "input_ids": input_encodings["input_ids"],
      "attention_mask": input_encodings["attention_mask"],
      "labels": target_encodings["input_ids"]

  }

In [12]:
# Preprocess the dataset
dataset_pt = dataset.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

# Train the model


In [13]:
# Set data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
# Trainer arguments
trainer_args = Seq2SeqTrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    fp16=True, learning_rate=2e-5
)

In [22]:
# Instantiate Trainer
trainer = Seq2SeqTrainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=data_collator,
                  train_dataset=dataset_pt['test'],
                  eval_dataset=dataset_pt['validation'])

In [23]:
# Train
trainer.train()

Step,Training Loss
10,1.976900
20,1.731300
30,1.900200
40,2.096900
50,1.228900
60,1.540800
70,1.629100
80,1.723400
90,1.764100
100,1.630700


TrainOutput(global_step=819, training_loss=1.7337002550434863, metrics={'train_runtime': 385.5373, 'train_samples_per_second': 2.124, 'train_steps_per_second': 2.124, 'total_flos': 314017624350720.0, 'train_loss': 1.7337002550434863, 'epoch': 1.0})

In [24]:
# Evaluate
def generate_batch_sized_chunks(list_of_elements, batch_size):
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i :i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device, column_text='article',
                                column_summary='highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
  score = metric.compute()
  return score

In [25]:
# Define metric

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')
bert_score_metric = load_metric('bertscore')

<ipython-input-25-29faf50bb172>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


In [26]:
# Evaluate model

score = calculate_metric_on_test_ds(
    dataset['validation'][0:100], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus model'] )

100%|██████████| 50/50 [02:02<00:00,  2.45s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus model,0.014242,0.0,0.014198,0.01412


In [27]:
score = calculate_metric_on_test_ds(
    dataset['validation'][0:100], bert_score_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

print(score)


100%|██████████| 50/50 [02:00<00:00,  2.41s/it]


AssertionError: ignored

In [28]:
## Save model and tokenizer
model.save_pretrained("pegasus-summarization-model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [29]:
!zip -r model.zip pegasus-summarization-model
!zip -r tokenizer.zip tokenizer

  adding: pegasus-summarization-model/ (stored 0%)
  adding: pegasus-summarization-model/config.json (deflated 60%)
  adding: pegasus-summarization-model/pytorch_model.bin (deflated 7%)
  adding: pegasus-summarization-model/generation_config.json (deflated 45%)
  adding: tokenizer/ (stored 0%)
  adding: tokenizer/tokenizer.json (deflated 78%)
  adding: tokenizer/special_tokens_map.json (deflated 82%)
  adding: tokenizer/spiece.model (deflated 50%)
  adding: tokenizer/tokenizer_config.json (deflated 78%)


In [31]:
from google.colab import files
files.download('model.zip')
#files.download('tokenizer.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [35]:
!cp /content/model.zip /content/drive/My\ Drive/